In [1]:
%pip install seaborn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="dark")
import warnings
warnings.filterwarnings("ignore")

In [5]:
import json
import random

In [6]:
!pip install more-itertools

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [7]:
import collections
import itertools
from collections import Counter
from more_itertools import take

import math
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam

from pathlib import Path
from sklearn.metrics import precision_recall_fscore_support

In [8]:
SEED = 1234

In [9]:
def set_seeds(seed=1234):
    """set seeds for reproducibility"""
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # For multi-gpu

In [10]:
set_seeds(SEED)


In [11]:
cuda = True
device = torch.device("cuda" if(
    torch.cuda.is_available() and cuda) else "cpu")
torch.set_default_tensor_type("torch.FloatTensor")

if device.type == "cuda":
    torch.set_default_tensor_type("torch.cuda.FloatTensor")

print(device)

cuda


In [12]:
train_df = pd.read_parquet("../../datasets/V1/train.parquet")
val_df = pd.read_parquet("../../datasets/V1/valid.parquet")
test_df = pd.read_parquet("../../datasets/V1/test.parquet")

In [13]:
# Feature and target
X_train = train_df.text.to_numpy()
y_train = train_df.rating

X_val =  val_df.text.to_numpy()
y_val = val_df.rating

X_test = test_df.text.to_numpy()
y_test = test_df.rating

In [14]:
print(f"X_train: {X_train.shape}, y_train:{y_train.shape}")
print(f"X_val: {X_val.shape}, y_val:{y_val.shape}")
print(f"X_test: {X_test.shape}, y_test:{y_test.shape}")

X_train: (31499,), y_train:(31499,)
X_val: (6750,), y_val:(6750,)
X_test: (6751,), y_test:(6751,)


In [15]:
print(f"Sample: {X_train[0]} -> {y_train[0]}")

Sample: python wrong codepython beginner trying execute following 
print( old enough vote please enter age )
input()

age 18

age 18 
 print( must 18 vote )

elif age 18 
 print ( voting age )


run program prints voting age matter low number input someone help sure really basic stuck 

thanks -> LQ_EDIT


In [16]:
# Label encoder to encode class labels
class LabelEncoder(object):
    """Encode labels into unqiue ids/integers"""
    def __init__(self, class_to_index={}):
        self.class_to_index = class_to_index or {}
        self.index_to_class = {v:k for k, v in self.class_to_index.items()}
        self.classes = list(self.class_to_index.keys())
        
    def __len__(self):
        return len(self.class_to_index)
    
    def __str__(self):
        return f"<LabelEncoder(num_classes={len(self)})>"
    
    def fit(self, y):
        classes = np.unique(y)
        for i, class_ in enumerate(classes):
            self.class_to_index[class_] = i
        self.index_to_class = {v:k for k, v in self.class_to_index.items()}
        self.classes = list(self.class_to_index.keys())
        return self
    
    def encode(self, y):
        encoded = np.zeros(len(y), dtype=int)
        for i, item in enumerate(y):
            encoded[i] = self.class_to_index[item]
        return encoded
    
    def decode(self, y):
        classes = []
        for i, item in enumerate(y):
            classes.append(self.index_to_class[item])
            
        return classes
    
    def save(self, fp):
        with open(fp, "w") as fp:
            contents = {"class_to_index": self.class_to_index}
            json.dump(contents, fp, indent=4, sort_keys=False)
    
    @classmethod
    def load(cls, fp):
        with open(fp, "r") as fp:
            kwargs = json.load(fp=fp)
        return cls(**kwargs)

In [17]:
label_encoder = LabelEncoder.load("../../artifacts/label_encoder.json")

In [18]:
# Encode all our labels
y_train = label_encoder.encode(y_train)

y_val = label_encoder.encode(y_val)

y_test = label_encoder.encode(y_test)

In [19]:
# Class weights
counts = np.bincount(y_train)
class_weights = {i: 1.0/count for i, count in enumerate(counts)}
print(f"counts: {counts}\n weights: {class_weights}")

counts: [10499 10500 10500]
 weights: {0: 9.524716639679969e-05, 1: 9.523809523809524e-05, 2: 9.523809523809524e-05}


In [20]:
# Tokenizer
class Tokenizer(object):
    def __init__(self, char_level, num_tokens=None, pad_token="<PAD>",
                oov_token="<UNK", token_to_index=None):
        self.char_level = char_level
        self.separator = "" if self.char_level else " "
        if num_tokens: num_tokens -=2 # pad token and unk token excluded
        self.num_tokens = num_tokens
        self.pad_token = pad_token
        self.oov_token = oov_token
        
        if not token_to_index:
            token_to_index = {pad_token: 0, oov_token: 1}
        self.token_to_index = token_to_index
        self.index_to_token = {v:k for k, v in self.token_to_index.items()}
        
    def __len__(self):
        return len(self.token_to_index)
    
    def __str__(self):
        return f"<Tokenizer(num_tokens={len(self)})>"
    
    def fit_on_texts(self, texts):
        if not self.char_level:
            texts = [text.split(' ') for text in texts]
        all_tokens = [token for text in texts for token in text]
        counts = Counter(all_tokens).most_common(self.num_tokens)
        self.min_token_freq = counts[-1][1]
        
        for token, count in counts:
            index = len(self)
            self.token_to_index[token] = index
            self.index_to_token[index] = token
        return self
    
    def texts_to_sequences(self, texts):
        sequences = []
        for text in texts:
            if not self.char_level:
                text = text.split(" ")
            sequence = []
            for token in text:
                sequence.append(self.token_to_index.get(
                    token, self.token_to_index[self.oov_token]
                ))
            sequences.append(sequence)
        return sequences
            
    def sequences_to_texts(self, sequences):
        texts = []
        for sequence in sequences:
            text = []
            for index in sequence:
                text.append(self.index_to_token.get(
                    index, self.oov_token
                ))
            texts.append(self.separator.join([token for token in text]))
        return texts
    
    def save(self, fp):
        with open(fp, "w") as fp:
            contents = {
                "char_level": self.char_level,
                "oov_token": self.oov_token,
                "token_to_index": self.token_to_index
            }
            json.dump(contents, fp, indent=4, sort_keys=False)
    
    @classmethod
    def load(cls, fp):
        with open(fp, "r") as fp:
            kwargs = json.load(fp=fp)
        return cls(**kwargs)

In [21]:
# Check the full vocabulary size of training data
def check_vocab_size(texts):
    texts = [text.split(' ') for text in texts]
    all_tokens = [token for text in texts for token in text]
    # print(len(all_tokens))
    vocab_counts = Counter(all_tokens).most_common(None)
    # print(vocab_counts)
    print(f"Total vocabulary size is: {len(vocab_counts)}")

In [22]:
check_vocab_size(X_train)

Total vocabulary size is: 243250


In [23]:
tokenizer = Tokenizer(char_level=False, num_tokens=5000)
tokenizer.fit_on_texts(texts=X_train)
VOCAB_SIZE = len(tokenizer)
print(tokenizer)

<Tokenizer(num_tokens=5000)>


In [24]:
tokenizer.save(Path("../../artifacts/rnn-tokenizer.json"))

In [25]:
import wandb

In [26]:
token_run = wandb.init(project="stackoverflow-quality", name="RNN-Tokenizer")
token_art = wandb.Artifact("RNN-Tokenizer", type="preprocessed_data", metadata=dict(num_tokens=5000))
token_art.add_file("../../artifacts/rnn-tokenizer.json")
token_run.log_artifact(token_art)

wandb: Currently logged in as: alokpadhi. Use `wandb login --relogin` to force relogin


In [27]:
# convert texts to sequences
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)
X_test = tokenizer.texts_to_sequences(X_test)

preprocessed_text = tokenizer.sequences_to_texts([X_train[0]])[0]
print("Text to indices: \n"
     f" (Preprocessed) -> {preprocessed_text}\n"
     f" (Tokenized) -> {X_train[0]}")

Text to indices: 
 (Preprocessed) -> python wrong <UNK beginner trying execute following <UNK old enough <UNK please enter age <UNK <UNK 18 
 print( must 18 <UNK <UNK age 18 
 print ( <UNK age <UNK program prints <UNK age matter low number input someone help sure really basic stuck 

thanks
 (Tokenized) -> [93, 241, 1, 1256, 72, 654, 88, 1, 899, 1586, 1, 95, 103, 742, 1, 1, 535, 3, 311, 422, 535, 1, 1, 742, 535, 3, 188, 22, 1, 742, 1, 164, 1187, 1, 742, 1760, 2134, 75, 62, 375, 53, 347, 439, 613, 1094, 900]


In [28]:
# Pad the sequences to max_length
def pad_sequences(sequences, max_seq_len=0):
    """Pad sequences to max_length in sequence"""
    # max_seq_len = max(max_seq_len, max(len(sequence) for sequence in sequences))
    # num_classes = sequences[0].shape[-1]
    padded_sequences = np.zeros((len(sequences), max_seq_len))
    for i, sequence in enumerate(sequences):
        if len(sequence) < max_seq_len:
            padded_sequences[i][:len(sequence)] = sequence
        else:
            # print(sequence)
            padded_sequences[i][:max_seq_len] = sequence[:200]
    return padded_sequences

In [29]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.y)

    def __str__(self):
        return f"<Dataset(N={len(self)})>"
    
    def __getitem__(self, index):
        X = self.X[index]
        y = self.y[index]
        return [X, len(X), y]
    
    def collate_fn(self, batch):
        batch = np.array(batch)
        X = batch[:, 0]
        seq_lens = batch[:, 1]
        y = batch[:, 2]
        
        X = pad_sequences(X, 200)
        
        seq_lens = np.array([seq_len if seq_len < 200 else 200 for seq_len in seq_lens])
        X = torch.LongTensor(X.astype(np.int32))
        seq_lens = torch.LongTensor(seq_lens.astype(np.int32))
        y = torch.LongTensor(y.astype(np.int32))
        
        return X, seq_lens, y
    
    def create_dataloader(self, batch_size, shuffle=False, drop_last=False):
        return torch.utils.data.DataLoader(
            dataset=self, batch_size=batch_size, collate_fn=self.collate_fn,
            shuffle=shuffle, drop_last=drop_last, pin_memory=True
        )

In [30]:

train_dataset = Dataset(X=X_train, y=y_train)
val_dataset = Dataset(X=X_val, y=y_val)
test_dataset = Dataset(X=X_test, y=y_test)

In [31]:
print("Dasets: \n"
     f"Train Dataset: {train_dataset.__str__()}\n"
     f"Val Dataset: {val_dataset.__str__()}\n"
     f"Test Dataset: {test_dataset.__str__()}\n"
     f"Sample Data:\n"
     f"X: {train_dataset[0][0]}\n"
     f"seq_len: {train_dataset[0][1]}\n"
     f"y {train_dataset[0][2]}")

Dasets: 
Train Dataset: <Dataset(N=31499)>
Val Dataset: <Dataset(N=6750)>
Test Dataset: <Dataset(N=6751)>
Sample Data:
X: [93, 241, 1, 1256, 72, 654, 88, 1, 899, 1586, 1, 95, 103, 742, 1, 1, 535, 3, 311, 422, 535, 1, 1, 742, 535, 3, 188, 22, 1, 742, 1, 164, 1187, 1, 742, 1760, 2134, 75, 62, 375, 53, 347, 439, 613, 1094, 900]
seq_len: 46
y 2


In [32]:
batch_size = 64
train_dataloader = train_dataset.create_dataloader(batch_size=batch_size)
val_dataloader = val_dataset.create_dataloader(batch_size=batch_size)
test_dataloader = test_dataset.create_dataloader(batch_size=batch_size)

In [33]:
batch_X, batch_seq_lens, batch_y = next(iter(test_dataloader))
print("Sample batch:\n"
     f"X: {list(batch_X.size())}\n"
     f"seq_lens: {list(batch_seq_lens.size())}\n"
     f"y: {list(batch_y.size())}\n"
     f"Sample point: \n"
     f"X: {batch_X[0]}\n"
     f"Seq_len: {batch_seq_lens[0]}\n"
     f"y: {batch_y[0]}")

Sample batch:
X: [64, 200]
seq_lens: [64]
y: [64]
Sample point: 
X: tensor([  55,  207, 1121,  609, 1298,  522,  457,  421,  511, 2226, 2641, 1271,
         184,    1,  207, 1121,  609, 1298,  522,  457,  421,  511, 2226, 2641,
        1271,  184,  522, 2464,  241,    1,   98,    1,    1,   13,  616,    3,
         620, 1746, 1544, 1026,    3,  176,    1,    3,  133,    1,    3,  700,
         351,    1,    3,  700,  297,    1,    3,  283,   61, 2564,    1,    1,
          11, 2090,    1,  190,  296,    1,   10,  189,    1,    1,    1,   50,
         359, 4075,    1,    1,    1,    3,  522, 2302, 1774, 1774, 1092,    3,
         283,  146, 2746,    3,  283,  361,  441,    3, 1087, 1006,    1,    3,
        2148, 1006,    1,    3, 1006,    1,    3,   13, 2954,  332,    1, 2737,
           1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,   

In [34]:
class Trainer(object):
    def __init__(self, model, device, loss_fn=None, optimizer=None, scheduler=None):
        self.model = model
        self.device = device
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.scheduler = scheduler
        
    def train_step(self, dataloader):
        self.model.train()
        loss = 0.0
        
        # iterate over batches
        for i, batch in enumerate(dataloader):
            batch = [item.to(device) for item in batch]
            inputs, targets = batch[:-1], batch[-1]
            self.optimizer.zero_grad()
            z = self.model(inputs)
            J = self.loss_fn(z, targets)
            J.backward()
            self.optimizer.step()
            
            loss += (J.detach().item() - loss) / (i+1)
        return loss
    
    def eval_step(self, dataloader):
        self.model.eval()
        loss = 0.0
        y_trues, y_probs = [], []
        
        with torch.inference_mode():
            for i, batch in enumerate(dataloader):
                batch = [item.to(device) for item in batch]
                inputs, y_true = batch[:-1], batch[-1]
                
                z = self.model(inputs)
                J = self.loss_fn(z, y_true).item()
                
                loss += (J - loss) / (i + 1)
                
                y_prob = F.softmax(z).cpu().numpy()
                y_probs.extend(y_prob)
                y_trues.extend(y_true.cpu().numpy())
                
            
            return loss, np.vstack(y_trues), np.vstack(y_probs)
        
    def predict_step(self, dataloader):
        self.model.eval()
        y_probs = []
        
        with torch.inference_mode():
            for i, batch in enumerate(dataloader):
                inputs, targets = batch[:-1], batch[-1]
                
                z = self.model(inputs)
                
                y_prob = F.softmax(z).cpu().numpy()
                y_probs.extend(y_prob)
            return np.vstack(y_probs)
    
    def train(self, num_epochs, patience, train_dataloder, val_dataloader, wandb_run):
        best_val_loss = np.inf
        for epoch in range(num_epochs):
            print(f"[INFO] Epoch: {epoch+1} training started")
            train_loss = self.train_step(dataloader=train_dataloader)
            print(f"[INFO] Epoch: {epoch+1} training finished")
            print(f"[INFO] Epoch: {epoch+1} evaluation started")
            val_loss, _, _ = self.eval_step(dataloader=val_dataloader)
            print(f"[INFO] Epoch: {epoch+1} evaluation finished")
            self.scheduler.step(val_loss)
            
            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model = self.model
                wandb_run.summary["Best val loss"] = best_val_loss
                _patience = patience
            else:
                _patience -= 1
            
            if not _patience:
                print("Stopping Early")
                break
            print(f"[INFO] Logging wandb")
            wandb.log({"train_loss": train_loss})
            wandb.log({"valid_loss": val_loss})
            
            print(
                f"Epoch: {epoch+1}\n"
                f"\t train_loss: {train_loss:.3f}, "
                f"\t val_loss: {val_loss: .3f}, "
                f"\t LR: {self.optimizer.param_groups[0]['lr']:.2E}, "
                f"_patience: {_patience}"
            )
        return best_model

In [35]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [36]:
class GRU(nn.Module):
    def __init__(self, embedding_dim, vocab_size, rnn_hidden_dim, hidden_dim, dropout, 
                 num_classes, pretrained_embeddings=None, freeze_embeddings=False, padding_idx=0):
        super(GRU, self).__init__()

        if pretrained_embeddings is None:
            self.embeddings = nn.Embedding(
                embedding_dim=embedding_dim, num_embeddings=vocab_size, padding_idx=padding_idx
            )
        else:
            pretrained_embeddings = torch.from_numpy(pretrained_embeddings).float()
            self.embeddings = nn.Embedding(
                embedding_dim=embedding_dim, num_embeddings=vocab_size,
                padding_idx=padding_idx, _weight=pretrained_embeddings
            )

        if freeze_embeddings:
            self.embeddings.weight.requires_grad = False

        self.gru = nn.GRU(embedding_dim, rnn_hidden_dim,
                         batch_first=True, bidirectional=True)

        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(rnn_hidden_dim*2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)
        
    def forward(self, inputs):
        x_in, seq_lens = inputs
        x_in = self.embeddings(x_in)
        # print(seq_lens.shape)
        packed_in = pack_padded_sequence(x_in, seq_lens.to('cpu'), batch_first=True, enforce_sorted=False)
        packed_out, h_n = self.gru(packed_in)
        
        z = torch.cat([h_n[0, :, :], h_n[1, :,:]], dim=1)
        
        z = self.fc1(z)
        z = self.dropout(z)
        z = self.fc2(z)
        
        return z

In [37]:
def load_glove_embeddings(embedding_file):
    embeddings = {}
    with open(embedding_file, "r") as fp:
        for index, line in enumerate(fp):
            values = line.split()
            word = values[0]
            embedding = np.asarray(values[1:], dtype='float32')
            embeddings[word] = embedding
    return embeddings

In [38]:
def make_embedding_matrix(embeddings, word_index, embedding_dim):
    embedding_matrix = np.zeros((len(word_index), embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [39]:
EMBEDDING_DIM = 100
RNN_HIDDEN_DIM = 128
HIDDEN_DIM = 100
NUM_CLASSES = len(label_encoder)

In [40]:
# Get glove pre-trained embeddings
# !wget https://nlp.stanford.edu/data/glove.6B.zip

In [41]:
embedding_file = "../../embeddings/glove.6B.{0}d.txt".format(EMBEDDING_DIM)
glove_embeddings = load_glove_embeddings(embedding_file=embedding_file)
embedding_matrix = make_embedding_matrix(embeddings=glove_embeddings, word_index=tokenizer.token_to_index,
                                        embedding_dim=EMBEDDING_DIM)
print(f"<Embeddings(words={embedding_matrix.shape[0]}, dim={embedding_matrix.shape[1]}))>")

<Embeddings(words=5000, dim=100))>


In [57]:
PRETRAINED_EMBEDDINGS = embedding_matrix
FREEZE_EMBEDDINGS = False

In [43]:
DROPOUT_P = 0.1

In [44]:
model = GRU(embedding_dim=EMBEDDING_DIM, vocab_size=VOCAB_SIZE,
           rnn_hidden_dim=RNN_HIDDEN_DIM, hidden_dim=HIDDEN_DIM,
           dropout=DROPOUT_P, num_classes=NUM_CLASSES, pretrained_embeddings=PRETRAINED_EMBEDDINGS,
            freeze_embeddings=FREEZE_EMBEDDINGS
           )
model = model.to(device)
print(model.named_parameters)

<bound method Module.named_parameters of GRU(
  (embeddings): Embedding(5000, 100, padding_idx=0)
  (gru): GRU(100, 128, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (fc1): Linear(in_features=256, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=3, bias=True)
)>


In [45]:
loss_fn = nn.CrossEntropyLoss()

In [66]:
LEARNING_RATE = 1e-4
PATIENCE = 4
NUM_EPOCHS = 50

In [47]:
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.1, patience=3
)

In [65]:
CONFIG = dict(
    learning_rate=LEARNING_RATE,
    patience=PATIENCE,
    epochs=NUM_EPOCHS,
    target_classes=NUM_CLASSES,
    optimizer="Adam",
    scheduler="ReduceLROnPlateau",
    loss_function="Cross Entropy",
    embedding="glove",
    embedding_dim=100,
    model="GRU",
)

In [67]:
run = wandb.init(project="stackoverflow-quality", config=CONFIG, name="RNN")

In [68]:
trainer = Trainer(model=model, device=device, loss_fn=loss_fn,
                 optimizer=optimizer, scheduler=scheduler)

In [69]:
best_model = trainer.train(
    NUM_EPOCHS, PATIENCE, train_dataloader, val_dataloader, run
)

[INFO] Epoch: 1 training started
[INFO] Epoch: 1 training finished
[INFO] Epoch: 1 evaluation started
[INFO] Epoch: 1 evaluation finished
[INFO] Logging wandb
Epoch: 1
	 train_loss: 0.156, 	 val_loss:  0.354, 	 LR: 1.00E-05, _patience: 4
[INFO] Epoch: 2 training started
[INFO] Epoch: 2 training finished
[INFO] Epoch: 2 evaluation started
[INFO] Epoch: 2 evaluation finished
[INFO] Logging wandb
Epoch: 2
	 train_loss: 0.155, 	 val_loss:  0.357, 	 LR: 1.00E-06, _patience: 3
[INFO] Epoch: 3 training started
[INFO] Epoch: 3 training finished
[INFO] Epoch: 3 evaluation started
[INFO] Epoch: 3 evaluation finished
[INFO] Logging wandb
Epoch: 3
	 train_loss: 0.151, 	 val_loss:  0.349, 	 LR: 1.00E-06, _patience: 4
[INFO] Epoch: 4 training started
[INFO] Epoch: 4 training finished
[INFO] Epoch: 4 evaluation started
[INFO] Epoch: 4 evaluation finished
[INFO] Logging wandb
Epoch: 4
	 train_loss: 0.150, 	 val_loss:  0.349, 	 LR: 1.00E-06, _patience: 4
[INFO] Epoch: 5 training started
[INFO] Epoch: 5

In [70]:
def get_metrics(y_true, y_pred, classes):
    performance = {"overall": {}, "class": {}}
    
    metrics = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    performance["overall"]["precision"] = metrics[0]
    performance["overall"]["recall"] = metrics[1]
    performance["overall"]["f1-score"] = metrics[2]
    performance["overall"]["num_samples"] = np.float64(len(y_true))
    
    # Per-class performance
    class_metrics = precision_recall_fscore_support(y_true, y_pred, average=None)
    for i in range(len(classes)):
        performance["class"][classes[i]] = {
            "precision": class_metrics[0][i],
            "recall": class_metrics[1][i],
            "f1-score": class_metrics[2][i],
            "num_samples": np.float64(class_metrics[3][i]),
        }
        
    return performance

In [71]:
test_loss, y_true, y_prob = trainer.eval_step(dataloader=test_dataloader)
y_pred = np.argmax(y_prob, axis=1)

In [72]:
performance = get_metrics(y_true, y_pred, classes=label_encoder.classes)
print(json.dumps(performance["overall"], indent=2))

{
  "precision": 0.8696557165854493,
  "recall": 0.8696489408976448,
  "f1-score": 0.8696316467728067,
  "num_samples": 6751.0
}


In [73]:
run.log({"precision": performance["overall"]["precision"], "recall": performance["overall"]["recall"], "f1-score": performance["overall"]["f1-score"]})

In [63]:
wandb.finish()

In [74]:
model_artifact = Path("../../model-artifacts/")

In [75]:
# save the model
torch.save(best_model.state_dict(), Path(model_artifact, "rnn-model.pt"))

In [76]:
model_art = wandb.Artifact(f"{wandb.run.name}_{wandb.run.id}", type="model", metadata=CONFIG)
model_art.add_file("../../model-artifacts/rnn-model.pt")
run.log_artifact(model_art)